In [30]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)
%matplotlib inline

import os
%config IPCompleter.greedy=True

from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder, PowerTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import ExtraTreeRegressor, DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor, BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.cluster import DBSCAN

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.impute import SimpleImputer, KNNImputer




In [2]:
os.chdir('D:\AIMLData\GL_Hackathon')


fd_train = pd.read_csv('frequency_domain_features_train.csv')
fd_test = pd.read_csv('frequency_domain_features_test.csv')
td_train = pd.read_csv('time_domain_features_train.csv')
td_test = pd.read_csv('time_domain_features_test.csv')
hr_train = pd.read_csv('heart_rate_non_linear_features_train.csv')
hr_test = pd.read_csv('heart_rate_non_linear_features_test.csv')

In [88]:
df3 = pd.merge(fd_train, td_train, how='outer', on='uuid')
df_train = pd.merge(df3, hr_train, how='outer', on='uuid')

df2 = pd.merge(fd_test, td_test, how='outer', on='uuid')
df_test = pd.merge(df2, hr_test, how='outer', on='uuid')

In [89]:
df_train.shape, df_test.shape

((369289, 37), (41033, 36))

In [56]:
# df_train = df_train.sample(36928)
# df_test = df_test.sample(4103)

In [57]:
# imputer = SimpleImputer(missing_values = 0, strategy='mean')
# df_train[['pNN25','pNN50']] = imputer.fit_transform(df_train[['pNN50','pNN50']])
# df_test[['pNN25','pNN50']] = imputer.transform(df_test[['pNN25','pNN50']])


In [90]:
lEncoder = LabelEncoder()
df_train['condition'] = lEncoder.fit_transform(np.array(df_train['condition']).reshape(-1,1))
df_test['condition'] = lEncoder.transform(np.array(df_test['condition']).reshape(-1,1))

C:\Users\NandhakumarSelvaraju\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


In [91]:
df_train.columns

Index(['uuid', 'VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT',
       'HF_NU', 'TP', 'LF_HF', 'HF_LF', 'MEAN_RR', 'MEDIAN_RR', 'SDRR',
       'RMSSD', 'SDSD', 'SDRR_RMSSD', 'HR', 'pNN25', 'pNN50', 'KURT', 'SKEW',
       'MEAN_REL_RR', 'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR',
       'SDSD_REL_RR', 'SDRR_RMSSD_REL_RR', 'KURT_REL_RR', 'SKEW_REL_RR', 'SD1',
       'SD2', 'sampen', 'higuci', 'datasetId', 'condition'],
      dtype='object')

In [92]:
df_train.drop(columns= ['datasetId'], inplace=True)
df_test.drop(columns= ['datasetId'], inplace=True)
df_train.drop(columns= ['uuid'], inplace=True)
y_id = df_test.pop('uuid')
df_train.drop(columns= ['SKEW_REL_RR','KURT_REL_RR'], inplace=True)
df_test.drop(columns= ['SKEW_REL_RR','KURT_REL_RR'], inplace=True)
df_train.drop(columns= ['SDSD_REL_RR'], inplace=True) 
df_test.drop(columns= ['SDSD_REL_RR'], inplace=True) 

In [93]:
# clusters = DBSCAN(eps = 100, n_jobs=-1).fit(df_train)
# df_train['DBCSANcluster'] = clusters.labels_
# df_train['DBCSANcluster'].value_counts()

In [64]:
# df_train = df_train[df_train['DBCSANcluster'] != -1]

In [94]:
df_train.shape, df_test.shape

((369289, 32), (41033, 31))

In [95]:
# y_train = df_train.pop('HR')
X_train = df_train.copy(deep = True)
X_test = df_test.copy(deep = True)

In [67]:
# X_train.drop(columns= ['DBCSANcluster'], inplace=True)

In [96]:
X_train.shape, X_test.shape, y_id.shape

((369289, 32), (41033, 31), (41033,))

In [97]:
cols = X_train.columns
cols

Index(['VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU',
       'TP', 'LF_HF', 'HF_LF', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD',
       'SDRR_RMSSD', 'HR', 'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR',
       'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDRR_RMSSD_REL_RR',
       'SD1', 'SD2', 'sampen', 'higuci', 'condition'],
      dtype='object')

In [98]:
pt = PowerTransformer()
pt_cols = ['VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU',
       'TP', 'LF_HF', 'HF_LF', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD',
       'SDRR_RMSSD', 'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR',
       'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDRR_RMSSD_REL_RR',
       'SD1', 'SD2', 'sampen', 'higuci']
X_train[pt_cols] = pt.fit_transform(X_train[pt_cols])
X_test[pt_cols] = pt.transform(X_test[pt_cols])

In [ ]:
model =  # k=>sqrt(n)

In [ ]:
model = StackingRegressor([
#     ('linReg', LinearRegression()),
#     ('lasReg', Lasso()),
#     ('ridReg', Ridge()),
#     ('dtrReg', DecisionTreeRegressor(min_samples_leaf = 10)),
#     ('etrReg', ExtraTreeRegressor(min_samples_leaf = 10)),
#     ('dtrReg2', DecisionTreeRegressor(min_samples_split = 10)),
    ('etrReg2', ExtraTreeRegressor(min_samples_split = 5)),
#     ('ranReg', RandomForestRegressor(n_estimators = 100, min_samples_leaf = 5, min_samples_split = 5, n_jobs=-1)),
    ('extReg', ExtraTreesRegressor(n_estimators = 100, min_samples_leaf = 5, min_samples_split = 5, n_jobs=-1)),
#     ('gboReg', GradientBoostingRegressor(n_estimators = 10, min_samples_leaf = 10, min_samples_split = 10)),
#     ('adaReg', AdaBoostRegressor(n_estimators = 10)),
    ('bagReg', BaggingRegressor(base_estimator = ExtraTreesRegressor(max_depth=25), n_jobs=-1)),
    ('knnReg', KNeighborsRegressor(n_neighbors = 10 , weights = 'distance'))
], n_jobs=-1)

In [ ]:
x = model.fit(X_train, y_train)

In [ ]:
y_test_predict = model.predict(X_test)
y_train_predict = model.predict(X_train)

In [ ]:
mean_absolute_error(y_train, y_train_predict)

In [ ]:
model.estimators[0][1]

# KNeighborsRegressor

In [142]:
from sklearn.model_selection import GridSearchCV 

X_t = X_train

cols = ['VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU',
       'TP', 'LF_HF', 'HF_LF', 'MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSSD', 'SDSD',
       'SDRR_RMSSD', 'pNN25', 'pNN50', 'KURT', 'SKEW', 'MEAN_REL_RR',
       'MEDIAN_REL_RR', 'SDRR_REL_RR', 'RMSSD_REL_RR', 'SDRR_RMSSD_REL_RR',
       'SD1', 'SD2', 'sampen', 'higuci', 'condition']

space = dict()
space['n_neighbors'] = range(3,10)
space['weights'] = ['uniform','distance']
space['p']=[1,2]

model = RandomizedSearchCV(KNeighborsRegressor(), 
                     param_distributions = space, 
                     scoring = make_scorer(mean_absolute_error, greater_is_better=False), 
                     n_iter= 10,
                     n_jobs= -1,
                     cv = 5)

model.fit(X_t[cols], X_t['HR'])

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
                   param_distributions={'n_neighbors': range(3, 10),
                                        'p': [1, 2],
                                        'weights': ['uniform', 'distance']},
                   scoring=make_scorer(mean_absolute_error, greater_is_better=False))

In [143]:
model.best_estimator_, model.best_score_, model.best_params_, model.cv_results_['mean_test_score'].mean()

(KNeighborsRegressor(n_neighbors=4, p=1, weights='distance'),
 -0.010110784192972636,
 {'weights': 'distance', 'p': 1, 'n_neighbors': 4},
 -0.026460478660026415)

In [148]:
y_test_predict = model.predict(X_test)


In [ ]:
model =  BaggingRegressor(n_estimators= 50,base_estimator = RandomForestRegressor(n_estimators = 100, n_jobs=-1), n_jobs=-1)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_test_predict = model.predict(X_test)
y_train_predict = model.predict(X_train)

In [ ]:
mean_absolute_error(y_train, y_train_predict)

In [ ]:
df = pd.concat([pd.DataFrame(y_id),pd.DataFrame(y_test_predict)], axis=1, ignore_index=True)
df.columns = ['uuid','HR']
df.to_csv('BGRSubmission.csv', index=False)

In [ ]:
for i in range(1,4):
    print(i)


# ExtraTreesRegressor

In [82]:
space = dict()
# space['n_estimators'] = [100]
# space['max_depth'] = range(3,15)
# space['min_samples_split']=range(3,15)
# space['bootstrap']=[True]

model = ExtraTreesRegressor(n_jobs=-1)

model.fit(X_train, pt.fit_transform(np.array(y_train).reshape(-1,1)))

<ipython-input-82-c8482192e1a5>:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, pt.fit_transform(np.array(y_train).reshape(-1,1)))


ExtraTreesRegressor(n_jobs=-1)

In [ ]:
# model.best_estimator_, model.best_score_, model.best_params_, model.cv_results_['mean_test_score'].mean()

In [83]:
y_test_predict = model.predict(X_test)
y_train_predict = model.predict(X_train)


In [85]:
mean_absolute_error(y_train, pt.inverse_transform(y_train_predict.reshape(-1,1)))

6.890926489826878e-12

In [86]:
y_test_predict = pt.inverse_transform(y_test_predict.reshape(-1,1))

# Stacking Regressor

In [ ]:
model = StackingRegressor([
    ('extReg', ExtraTreesRegressor(bootstrap=True, max_depth=14, min_samples_split=4, n_jobs=-1)),
    ('knnReg', KNeighborsRegressor(n_neighbors = 8, p = 1))
], n_jobs=-1)

model.fit(X_train, y_train)

In [145]:
y_test_predict = model.predict(X_test)
y_train_predict = model.predict(X_train)


ValueError: query data dimension must match training data dimension

In [ ]:
mean_absolute_error(y_train, y_train_predict)

In [149]:
df = pd.concat([pd.DataFrame(y_id),pd.DataFrame(y_test_predict)], axis=1, ignore_index=True)
df.columns = ['uuid','HR']
df.to_csv('Submission.csv', index=False)
